- Global variables bind inside the same module
ex.

In file `spam.py`:
```python
x = 42

def blah():
	print(x)
```   

In file `egg.py`:
```python
from spam import blah

x = 10

blah() #10

blash.__globals__['x'] #42
```


- When a module is imported, __all of the statements in the module execute__ one after another until the end of file is reached.  



- import always executes the __entire__file




- modules only load once, then it's cached. You can check by:
```python
import sys
'math' in sys.modules # whether math is loaded
```

In [14]:
import sys
import math

print('math' in sys.modules)

True


- So how can we reload a module
> Deleting it from sys.modules? Will that do the trick?

In [15]:
del sys.modules['math'] 
print('math' in sys.modules)
math.sin(2)

False


0.9092974268256817

- Nope, we need another library importlib

In [17]:
import importlib
import math 

importlib.reload(math)

<module 'math' from '/home/maxliu/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-x86_64-linux-gnu.so'>

- For large collections of code, it is usually desirable to organize modules into a hierarchy:

```
spam/
    foo.py
    bar/
        grok.py
```

- To do it, you just add \__init\__.py files

```
spam/
    __init__.py
    foo.py
    bar/
        __init__.py
        grok.py
```

- Now you can use multiple level import ex. `import spam.foo`

- Don't use implicit relative imports in packages:

```python
spam/
    __init__.py
    foo.py     # has a class named Foo
    bar.py     # has a class named Bar

# bar.py
import foo  # Relative import of foo, works only in python2

# right way to do it:
from . import foo  # Import from same level
from .. import foo # Loads ../foo.py
from ..grok import foo # Loads ../grok/foo.py
```




- What is \__init\__.py file??
> They should mainly be used to stitch together multiple source files into a "unified" top-level import.

```python
# in __init__.py
from .foo import Foo
from .bar import Bar
```  

- Now users see a single unified top-level package

```python
import spam

f = spam.Foo()
b = spam.Bar()
```

#### Controlling Exports
- each submodule should define \__all\__
- controls behaviour of `from module import *`

```python
# foo.py
__all__ = ['Foo']

#bar.py
__all__ = ['Bar']

# This allows easy combination in __init__.py
#__init__.py
from .foo import *
from .bar import *

__all__ = (foo.__all__ + bar.__all__)
```

In [ ]:
# An export decorator
def export(defn):
    # register function in the globals() scope
    globals()[defn.__name__] = defn
    # append it to __all__
    __all__.append(defn._name)
    return defn

#now defining functions to be exported
@export
def foo():
    pass

- Hacking \__path\__

```Python
# modeuls has a __path__ attribute
import xml
xml.__path__  # ['/home/maxliu/anaconda3/lib/python3.6/xml']

xml.__path__.append('./') # appending current directory into the path

import xml.spams  # now suddenly spams is also part of xml library


```



- A package can "upgrade" itself on import 

```python
# xml/__init__.py

try:
    import _xmlplus
    import sys
    sys.modules[__name__] = _xmlplus
except ImportError:
    pass
```

#### The "-m" option

- Makes the Python version explicit

`python3 -m pip install package`  __vs.__ `pip install package`



- \__main\__.py designates main for a package

- Also makes a pacakge directory executable

```python
spam/
    __init__.py
    __main__.py
    foo.py
    bar.py
    
python3 -m spam  # Run package as main
```

- Some other tools:
```bash
python3 -m profile some.py
python3 -m pdb some.py
python3 -m coverage run some.py
python3 -m zipfile -c some.zip some/*.py  # kinda cool
```



- Fun fact: you can prepend a zipfile with #! to make it executeable like a script (the zipfile shoudl have \__main\__.py of course

```bash
python3 -m zipfile -c spamzip spam/*.py
echo -e '#!/usr/bin/env python3\n' > spamapp
cat spam.zip >> spamapp
chmod +x spamapp
./spamapp
```

#### sys.path

In [39]:
import sys

print(sys.path)

# sys.prefix is the base location of python
print(sys.prefix)

# exec_prefix is the location of compiled binaries(C)
print(sys.exec_prefix)

['', '/home/maxliu/anaconda3/lib/python36.zip', '/home/maxliu/anaconda3/lib/python3.6', '/home/maxliu/anaconda3/lib/python3.6/lib-dynload', '/home/maxliu/anaconda3/lib/python3.6/site-packages', '/home/maxliu/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/maxliu/.ipython']
/home/maxliu/anaconda3
/home/maxliu/anaconda3


#### Python standard libraries usually located at:
- sys.prefix + '/lib/python3X.zip
- sys.prefix + '/lib/python3.X
- sys.prefix + '/lib/python3.X/plat-sysname'
- sys.exec_prefix + '/lib/python3.X/lib-dynload

 #### PYTHONHOME magic
 
 ```bash
 cd /tmp
 mkdir lib
 env PYTHONHOME=. python3    # crashes, python can't open from current location
 
 cp -R /home/maxliu/anaconda3/lib ./lib # copy libraries into current directory
 env PYTHONHOME=. python3    # Boom! it works
 ```
 
 This is kinda of how virtualenv works!

#### Virtual Environments

- makes a python virtual environment

```bash
python3 -m venv spam

# creates the director structure
spam/
    pyvenv.cfg
    bin/
        activate
        easy_install
        pip 
        python3
    inlude/
        ...
    lib/
        python3.6/
            site-packages/
                ....
```


#### Namespace
- Omit \__init\__.py and you get a "namespace"

```python
spam/
    foo.py
    bar.py
    
import spam
spam     # <module 'grok' (namespace)>
```

- Now if there's two spam directory in different locations, and both locations are on sys.path, then all the files under both spam directories goes into the same package.

#### What is a module


- In python, module is an object!!

In [2]:
import math

type(math)

module

In [18]:
#creating a module object
import types

mod = types.ModuleType('spam')
mod

<module 'spam'>

In [23]:
# It's just a dictionary
mod.__dict__

{'__doc__': None,
 '__loader__': None,
 '__name__': 'spam',
 '__package__': None,
 '__spec__': None}

In [22]:
#bonus: using a immutable dictionary
d = {'a':1, 'b':2}

frozen_dict = types.MappingProxyType(d)
frozen_dict['c'] = 3

TypeError: 'mappingproxy' object does not support item assignment

In [ ]:
# a minimal implementation of import
import types

def import_module(modname):
    sourcepath = modname + '.py'
    with open(sourcepath, 'r') as f:
        sourcecode = f.read()
        
    mod = types.ModuleType(modname)
    mod.__file__ = sourcepath
    code = compile(sourcecode, sourcepath, 'exec')
    exec(code, mod.__dict__)
    return mod

In [31]:
# compile() function compiles the python code into low level byte codes
code = 'for i in range(5): print(i)'
code = compile(code, 'code.py', 'exec')

import dis
dis.dis(code) # dissemble the code

  1           0 SETUP_LOOP              24 (to 26)
              2 LOAD_NAME                0 (range)
              4 LOAD_CONST               0 (5)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                12 (to 24)
             12 STORE_NAME               1 (i)
             14 LOAD_NAME                2 (print)
             16 LOAD_NAME                1 (i)
             18 CALL_FUNCTION            1
             20 POP_TOP
             22 JUMP_ABSOLUTE           10
        >>   24 POP_BLOCK
        >>   26 LOAD_CONST               1 (None)
             28 RETURN_VALUE


- cyclic imports in package will crash, because the reference to a submodule only get created after the entire submodule imports

In [5]:
# custimizing your import function

import builtins

def my_import(name, *args, builtin_imp=__import__):
    print('Importing', name)
    mod = builtin_imp(name, *args)
    print('Module path:', mod.__file__)
    print('Module doc:', mod.__doc__)
    return mod
    
builtins.__import__ = my_import

In [6]:
import math

Importing session
Module path: /home/maxliu/anaconda3/lib/python3.6/site-packages/jupyter_client/session.py
Module doc: Session object for building, serializing, sending, and receiving messages.

The Session object supports serialization, HMAC signatures,
and metadata on messages.

Also defined here are utilities for working with Sessions:
* A SessionFactory to be used as a base class for configurables that work with
Sessions.
* A Message object for convenience that allows attribute-access to the msg dict.

Importing math
Module path: /home/maxliu/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-x86_64-linux-gnu.so
Module doc: This module is always available.  It provides access to the
mathematical functions defined by the C standard.


#### Subclass Module

In [4]:
import types
